In [1]:
!pip install jupyter-dash
!pip install -U plotly
!pip install Dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 11.8 MB/s 
     |████████████████████████████████| 1.6 MB 47.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 973 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import pandas as pd
import plotly.express as px
import pandas as pd
import plotly
import numpy as np
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import folium
import scipy as scp
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import sklearn
import seaborn as sns
import matplotlib.dates as md
from dash import Dash


In [ ]:
# Fazer o input do dataset
from google.colab import files
uploaded = files.upload()

In [45]:
dataset = 'tsunami_dataset.csv'
tsunamis = pd.read_csv(dataset)

tsunamisFrom1800 = tsunamis.query("YEAR > 1800")

allTsunamisEvents = tsunamisFrom1800.dropna(subset=['EVENT_VALIDITY'])
allTsunamisEvents = allTsunamisEvents.groupby('EVENT_VALIDITY').agg({'EVENT_VALIDITY': 'count'})
allTsunamisEvents = allTsunamisEvents.rename(index=str, columns={"EVENT_VALIDITY": "EVENT_VALIDITY", "EVENT_VALIDITY": "VALUES"})
allTsunamisEvents = allTsunamisEvents.sort_values(by='VALUES', ascending=False)

# tsunamis de verdade
definiteTsunamis = tsunamisFrom1800[tsunamisFrom1800['EVENT_VALIDITY'].str.contains('Definite Tsunami')]

# retirada de valores nulos na descricao do dano dos tsunamis
damageTsunamis = tsunamisFrom1800.dropna(subset=['DAMAGE_TOTAL_DESCRIPTION'])

# pegar so os tsunamis com danos extremos
extremeDamageTsunamis = damageTsunamis[damageTsunamis['DAMAGE_TOTAL_DESCRIPTION'].str.contains('Extreme')]

# tsunamis de danos extremos agrupados por causa
damageTsunamisPerCause = extremeDamageTsunamis.groupby('CAUSE').agg({'DAMAGE_TOTAL_DESCRIPTION': 'count'})
damageTsunamisPerCause = damageTsunamisPerCause.rename(index=str, columns={"DAMAGE_TOTAL_DESCRIPTION": "DAMAGE_TOTAL_DESCRIPTION", "DAMAGE_TOTAL_DESCRIPTION": "VALUES"})
damageTsunamisPerCause = damageTsunamisPerCause.sort_values(by='VALUES', ascending=False)

# tsunamis por ano agrupados
tsunamisPerYear = tsunamisFrom1800.groupby('YEAR').agg({'COUNTRY': 'count'})
tsunamisPerYear = tsunamisPerYear.rename(index=str, columns={"COUNTRY": "COUNTRY", "COUNTRY": "VALUES"})
# tsunamis por ano agrupados em ordem crescente pelo ano que mais teve
sortedTsunamisPerYear = tsunamisPerYear.sort_values(by='VALUES', ascending=False)

# tsunamis agrupados por pais
tsunamisPerCountry = tsunamisFrom1800.groupby('COUNTRY').agg({'COUNTRY': 'count'})
tsunamisPerCountry = tsunamisPerCountry.rename(index=str, columns={"CONTRY": "COUNTRY", "COUNTRY": "VALUES"})
# tsunamis por pais agrupados em ordem crescente
sortedTsunamisPerCountry = tsunamisPerCountry.sort_values(by='VALUES', ascending=False)
sortedTsunamisPerCountry['CNT'] = sortedTsunamisPerCountry.index

# pegar os tsunamis com mais mortes
tsunamisDeaths = tsunamisFrom1800.dropna(subset=['DEATHS_TOTAL_DESCRIPTION'])
tsunamisDeaths = tsunamisDeaths[tsunamisDeaths['DEATHS_TOTAL_DESCRIPTION'].str.contains('Very Many')]
tsunamisDeaths = tsunamisDeaths.groupby('REGION').agg({'DEATHS_TOTAL_DESCRIPTION': 'count'})
tsunamisDeaths = tsunamisDeaths.sort_values(by='DEATHS_TOTAL_DESCRIPTION', ascending=False)
tsunamisDeaths = tsunamisDeaths.rename(index=str, columns={"DEATHS_TOTAL_DESCRIPTION": "Qnt mais de 1000 mortos",})

!mkdir assets

# grafico dos mais letais
mostLetal = tsunamisDeaths.plot(kind='bar')
plt.title("Região dos tsunamis mais letais")
plt.xlabel("Região")
plt.savefig('assets/most_letal.png', bbox_inches='tight')

# grafico de causa dos tsunamis de danos extremos
damagePerCause = damageTsunamisPerCause.plot(kind='bar')
plt.title("Causa dos tsunamis de danos extremos")
plt.xlabel("Causa")
plt.ylabel("Quantidade")
plt.savefig('assets/damage_per_cause.png', bbox_inches='tight')

# grafico dos tsunamis por ano em linha
tsunamisPerYear.groupby('YEAR').mean().plot()
plt.title("Tsunamis por ano")
plt.xlabel("Anos")
plt.ylabel("Quantidade")
plt.savefig('assets/tsunamis_per_year.png', bbox_inches='tight')

# grafico dos paises com mais tsunamis
df2 = sortedTsunamisPerCountry[:10].copy()
new_row = pd.DataFrame(data = {
    'CNT' : ['OUTROS'],
    'VALUES' : [sortedTsunamisPerCountry['VALUES'][10:].sum()]
})
df2 = pd.concat([df2, new_row])
def my_autopct(pct):
    return ('%.2f' % pct) if pct > 3 else ''
piePlot = df2.plot(kind='pie', y='VALUES', legend=False, autopct=my_autopct, radius=2.5, labels = df2['CNT'])
plt.savefig('assets/tsunamis_per_country.png', bbox_inches='tight')

# grafico todos tipo de tsunamis confirmados e não confirmados
confirmedTsunamis = allTsunamisEvents.plot(kind='bar')
plt.title("Eventos de tsunamis confirmados e dúvidas")
plt.xlabel("Evento")
plt.ylabel("Quantidade")
plt.savefig('assets/confirmed_and_not_confirmed_tsunamis.png', bbox_inches='tight')

ds_mapa = definiteTsunamis[['COUNTRY', 'LOCATION_NAME', 'LONGITUDE', 'LATITUDE', 'CAUSE']].drop_duplicates()

m = folium.Map(location=[0, 0], zoom_start=2, tiles="Stamen Terrain")

for i in range(len(ds_mapa)):
  tsunami = ds_mapa.iloc[i]

  folium.Marker(
    location=[tsunami['LATITUDE'], tsunami['LONGITUDE']],
    tooltip=tsunami['LOCATION_NAME'],
    popup= folium.Popup("<b> PAÍS: " + str(tsunami['COUNTRY']) + "</b>" +
                        "<br> LOCALIZAÇÃO: " + str(tsunami['LOCATION_NAME']) +
                        "<br> CAUSA: " + str(tsunami['CAUSE']),
                        min_width=300, max_width=300
                       ),
    icon=folium.Icon(color="red", icon="info-sign"),
  ).add_to(m)

m.save('map.html')




/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.



b'Skipping line 29: expected 1 fields, saw 367\nSkipping line 85: expected 1 fields, saw 3\nSkipping line 130: expected 1 fields, saw 3\nSkipping line 194: expected 1 fields, saw 3\nSkipping line 203: expected 1 fields, saw 6\nSkipping line 235: expected 1 fields, saw 4\nSkipping line 248: expected 1 fields, saw 4\nSkipping line 261: expected 1 fields, saw 4\nSkipping line 274: expected 1 fields, saw 4\nSkipping line 287: expected 1 fields, saw 4\nSkipping line 300: expected 1 fields, saw 4\nSkipping line 313: expected 1 fields, saw 4\nSkipping line 326: expected 1 fields, saw 4\nSkipping line 343: expected 1 fields, saw 4\nSkipping line 350: expected 1 fields, saw 4\nSkipping line 359: expected 1 fields, saw 4\nSkipping line 368: expected 1 fields, saw 4\nSkipping line 393: expected 1 fields, saw 4\nS

UndefinedVariableError: ignored

In [41]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    
    html.H3('Tema do projeto', style={'fontFamily': 'Arial'}),
    html.Div(html.P("O tema escolhido busca trazer os tsunamis ocorridos nos " +
     "últimos anos em todo o mundo e assim proporcionar uma visão global e diferente mostrando os locais onde ocorreram tsunamis tais quais suas causas, " +
      "danos, mortes e ano buscando passar uma visão geral dos locais mais atingidos e mais perigosos para se viver onde podem acontecer este fenômeno. " +
       "Um tsunami é uma série de ondas de água causada pelo deslocamento de um grande volume de um corpo de água, como um oceano ou um grande lago", style={'fontFamily': 'Arial'})),

    html.Br(),

    html.H3('Resultados obitos', style={'fontFamily': 'Arial'}),
    html.Div(html.P("O dataset utilizado  é da The Global Historical Tsunami Database mantido pela NCEI/WDS americana e conta com 2400 registros de tsunamis ocorridos entre 2100 a.c até 2020 " +
     "pelos oceanos Atlântico, Índico, Pacífico e mares caribenhos e mediterrâneo, este dataset possui como informações mais relevantes de cada tsunami o ano, longitude e latitude, país, " +
      "região, causa, mortes e danos causados entre outros dados além de identificar aquele registro/evento como um tsunami definitivo ou tsunamis que não foram confirmados, " +
       "para o uso do dataset transformamos para pegarmos apenas os registro do ano 1800 até os dias de hoje para registros de melhor qualidade e mais informações, o dataset foi " +
        "separado com tsunamis dados como verdadeiros, tsunamis de danos extremos agrupando pelas causas destes tsunamis para obter uma relação entre os danos e as causas para saber qual " +
         "causa de tsunami é mais grave, tsunamis causados por ano e por país tal qual uma separação do dataset em mortes causadas pelos tsunamis e agrupando quais " +
          "as regiões que obtiveram mais tsunamis com mais de mil mortes.", style={'fontFamily': 'Arial', 'textJustify': 'inter-word'})),
    html.Div(html.P("Assim o resultado obtido através das transformações e gráficos feitos percebemos que a maioria dos tsunamis são causados por terremotos e dessa maneira os países " +
     "mais afetados são os países que estão próximos a falhas geográficas e ao círculo de fogo do pacífico, locais mais espalhados e próximos aos polos norte e sul " +
      "quase sempre possuem tsunamis são gerados por deslizamentos de terra ou gelo que movimentam a água do oceano, também nota-se que as regiões mais afetadas " +
       "com mortes e danos pelos tsunamis são ilhas no pacifico atingidas por terremotos, outra métrica interessante é que os dados obtidos durantes os anos nem sempre são " +
        "confirmados como tsunamis verdadeiros muitos dos dados não apresentam confirmação do que realmente houve naquele local.", style={'fontFamily': 'Arial', 'textJustify': 'inter-word'})),

    html.Br(),
    html.Br(),

    html.Div([
      html.Img(src="assets/confirmed_and_not_confirmed_tsunamis.png", style={'marginRight': '32px'}),
      html.Img(src="assets/damage_per_cause.png", style={'marginRight': '32px'}),
      html.Img(src="assets/most_letal.png", style={'marginRight': '32px'}),
      html.Img(src="assets/tsunamis_per_year.png"),
      html.Br(),
      html.Br(),
      html.Br(),
      html.Br(),
      html.H4("Tsunamis por país em relação ao geral em %:", style={'fontFamily': 'Arial'}),
      html.Img(src="assets/tsunamis_per_country.png"),
    ]),
    

    html.Br(),
    html.Br(),

    html.H4("Localização dos tsunamis confirmados entre 1800 e 2020:", style={'fontFamily': 'Arial'}),
    html.Iframe(id='map', srcDoc=open('map.html', 'r').read(), width='100%', height='600'),
    
])

# Run app and display result inline in the notebook
app.run_server(mode='external', debug=True)

Dash app running on:


<IPython.core.display.Javascript object>